In [11]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.preprocessing import image

# Directorio de las imágenes organizadas en 'Benign' y 'Malignant'
train_dir = 'test/Benign'  # Asegúrate de que aquí tienes las carpetas 'Benign' y 'Malignant'

# Preprocesamiento de imágenes con aumentación y escalado
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Cambia el tamaño si es necesario
    batch_size=32,
    class_mode='binary',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation')

# Definir el modelo CNN
model = models.Sequential()

# Capas de convolución y pooling
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Aplanar las salidas y añadir capas densas
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  # Salida binaria (benigno o maligno)

# Compilar el modelo
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenamiento del modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32)

# Guardar el modelo entrenado para usarlo más tarde
model.save('skin_lesion_classifier.h5')

# Evaluación con nuevas imágenes
def predict_image_class(image_path):
    # Cargar y preprocesar la nueva imagen
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Realizar la predicción
    prediction = model.predict(img_array)

    # Mostrar resultado
    if prediction < 0.5:
        print("La imagen parece ser una lesión BENIGNA.")
    else:
        print("La imagen parece ser una lesión MALIGNA.")

# Prueba con una imagen nueva
predict_image_class('12.jpg')  # Aquí debes poner la ruta de la imagen que quieras predecir


FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'test/Benign'